In [1]:
!pip install datasets transformers
!pip install ipython-autotime

%load_ext autotime

time: 2.76 ms (started: 2022-05-30 02:12:01 +00:00)


In [2]:
! nvidia-smi
! pwd && ls -alh

from os import path
NAMESPACE = "hf_script_expriments"
NAMESPACE_DIR = path.abspath(path.join(path.curdir, NAMESPACE))
NAMESPACE_DIR

Mon May 30 02:12:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

'/content/hf_script_expriments'

time: 233 ms (started: 2022-05-30 02:12:01 +00:00)


In [23]:
from datasets import load_dataset

def build_corpus():
  corpus = load_dataset(
      'csv', 
      data_files=[
        "/content/datasets/undp-jobs-load-ready.csv",
        "/content/datasets/monster-jobs-load-ready.csv",
        "/content/datasets/dice-jobs-load-ready.csv"
      ], 
      column_names=["text"]
  )
  corpus = corpus["train"].remove_columns('__index_level_0__').train_test_split(test_size=0.1)
  return corpus

full_corpus = build_corpus()
full_corpus_path = "./datasets/full_corpus"
full_corpus.save_to_disk(full_corpus_path)

Using custom data configuration default-9cf3073f1433db7c
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-9cf3073f1433db7c/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Flattening the indices:   0%|          | 0/133 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/15 [00:00<?, ?ba/s]

time: 5.08 s (started: 2022-05-30 05:02:53 +00:00)


In [27]:
full_corpus

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 132796
    })
    test: Dataset({
        features: ['text'],
        num_rows: 14756
    })
})

time: 6.98 ms (started: 2022-05-30 05:10:27 +00:00)


In [34]:
!TF_GPU_ALLOCATOR=cuda_malloc_async python /content/transformers/examples/tensorflow/language-modeling/run_mlm.py \
  --model_name_or_path distilbert-base-cased \
  --output_dir domain-adapted-distillbert-20btch-30bp_mlm \
  --dataset_name /content/datasets/full_corpus \
  --max_seq_length 2000 \
  --per_device_train_batch_size 20 \
  --mlm_probability 0.30

2022-05-30 10:20:20.784956: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2022-05-30 10:20:20.785023: E tensorflow/core/common_runtime/gpu/gpu_process_state.cc:69] TF_GPU_ALLOCATOR=cuda_malloc_async environment found, but TensorFlow was not compiled with CUDA 11.2+.
Using custom data configuration default-189e88e56bbd583c
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-189e88e56bbd583c/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)
100% 1/1 [00:00<00:00, 589.92it/s]
Loading cached split indices for dataset at /root/.cache/huggingface/datasets/csv/default-189e88e56bbd583c/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-a5d3cb00b77900a9.arrow and /root/.cache/huggingface/datasets/csv/default-189e88e56bbd583c/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83

In [42]:
!zip -r domain-adapted-distillbert-20btch-30bp_mlm.zip /content/domain-adapted-distillbert-20btch-30bp_mlm

  adding: content/domain-adapted-distillbert-20btch-30bp_mlm/ (stored 0%)
  adding: content/domain-adapted-distillbert-20btch-30bp_mlm/config.json (deflated 43%)
  adding: content/domain-adapted-distillbert-20btch-30bp_mlm/tf_model.h5 (deflated 8%)
time: 34.2 s (started: 2022-05-30 18:55:15 +00:00)


In [43]:
!du -h domain-adapted-distillbert-20btch-30bp_mlm.zip

312M	domain-adapted-distillbert-20btch-30bp_mlm.zip
time: 181 ms (started: 2022-05-30 18:55:49 +00:00)
